In [1]:
#
#Tweets Machine Learning Coursework 
#COMP3222
#

# To support both python 2 and 3
from __future__ import division, print_function, unicode_literals

# Common imports
import os
import numpy as np
import pandas as pd

# These two lines are required to use Tensorflow 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# To plot nice figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Clear tensorflow's and reset seed
def reset_graph(seed=None):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)



Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#get data
def getData(file_name):
    dict_data = {}
    data_file = open(file_name, "r", encoding="utf8")
    raw_data_txt = data_file.readlines()
    raw_data = []
    for data in raw_data_txt:
        raw_data.append(data.split("\t"))
    raw_data.pop(0)

    target_text = [data[6] for data in raw_data]
    dict_data['text'] = np.array([tweet[1] for tweet in raw_data])
    dict_data['imageIds'] = np.array([tweet[3] for tweet in raw_data])
    dict_data['timestamp'] = np.array([tweet[5] for tweet in raw_data])
    dict_data['label'] = np.array([1 if target=='real' or target =='real\n' else 0 for target in target_text])

    return dict_data

df_train = pd.DataFrame.from_dict(getData("mediaeval-2015-trainingset.txt"))
df_test = pd.DataFrame.from_dict(getData("mediaeval-2015-testset.txt"))
print("Done")


Done


In [4]:
df_train

,text,imageIds,timestamp,label
0,¿Se acuerdan de la película: “El día después d...,sandyA_fake_46,Mon Oct 29 22:34:01 +0000 2012,0
1,@milenagimon: Miren a Sandy en NY! Tremenda i...,sandyA_fake_09,Mon Oct 29 19:11:23 +0000 2012,0
2,"Buena la foto del Huracán Sandy, me recuerda a...",sandyA_fake_09,Mon Oct 29 18:11:08 +0000 2012,0
3,Scary shit #hurricane #NY http://t.co/e4JLBUfH,sandyA_fake_29,Mon Oct 29 19:15:33 +0000 2012,0
4,My fave place in the world #nyc #hurricane #sa...,sandyA_fake_15,Mon Oct 29 20:46:02 +0000 2012,0
...,...,...,...,...
14478,@BobombDom *slaps TweetDeck with the PigFish h...,pigFish_01,Tue Mar 11 03: 48: 36 +0000 2014,0
14479,New Species of Fish found in Brazil or just Re...,pigFish_01,Mon Mar 10 18: 09: 26 +0000 2014,0
14480,What do we call this? #pigFISH http: \/\/t.co\...,pigFish_01,Mon Mar 10 10: 59: 45 +0000 2014,0
14481,Pigfish ? E dopo il pescecane c'è il pesce mai...,pigFish_01,Sun Mar 09 20: 07: 10 +0000 2014,0


In [5]:
#Need to run get data always before re running this one
#tokenize train data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_train['text'])
df_train['text'] = tokenizer.texts_to_sequences(df_train['text'])
max_tokens = map(max, df_train['text'].values[:])
biggest_token = max(max_tokens)
MAX_TWEET_LENGTH = 30

#tokenize test data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_test['text'])
df_test['text'] = tokenizer.texts_to_sequences(df_test['text'])
MAX_TWEET_LENGTH = 30

In [6]:
df_train

,text,imageIds,timestamp,label
0,"[7198, 7199, 16, 23, 809, 2719, 1108, 783, 16,...",sandyA_fake_46,Mon Oct 29 22:34:01 +0000 2012,0
1,"[4378, 1165, 9, 4, 39, 25, 4379, 165, 72, 48, ...",sandyA_fake_09,Mon Oct 29 19:11:23 +0000 2012,0
2,"[833, 23, 66, 72, 48, 4, 90, 4377, 9, 23, 809,...",sandyA_fake_09,Mon Oct 29 18:11:08 +0000 2012,0
3,"[137, 84, 6, 25, 3, 1, 2, 7203]",sandyA_fake_29,Mon Oct 29 19:15:33 +0000 2012,0
4,"[38, 4381, 479, 8, 5, 252, 15, 6, 4, 102, 869,...",sandyA_fake_15,Mon Oct 29 20:46:02 +0000 2012,0
...,...,...,...,...
14478,"[28398, 28399, 28400, 71, 5, 1489, 3, 1, 2, 28...",pigFish_01,Tue Mar 11 03: 48: 36 +0000 2014,0
14479,"[14, 7197, 7, 2379, 92, 8, 28402, 282, 80, 220...",pigFish_01,Mon Mar 10 18: 09: 26 +0000 2014,0
14480,"[89, 156, 107, 644, 10, 1489, 3, 1, 2, 28405]",pigFish_01,Mon Mar 10 10: 59: 45 +0000 2014,0
14481,"[1489, 402, 2955, 1770, 28406, 28407, 1770, 28...",pigFish_01,Sun Mar 09 20: 07: 10 +0000 2014,0


In [6]:
#Find out word count of texts.
X_lens = [len(x.split(' ')) for x in df_train['text'].values]
X_lens = np.array(X_lens)
plt.hist(X_lens, 50)
plt.show()
x = np.unique((X_lens >= MAX_TWEET_LENGTH), return_counts=True)
#Length most under 30 in length

AttributeError: 'list' object has no attribute 'split'

In [7]:
X_train_text = tf.keras.preprocessing.sequence.pad_sequences(df_train['text'].values, maxlen=MAX_TWEET_LENGTH, padding='post', truncating='post')
X_test_text = tf.keras.preprocessing.sequence.pad_sequences(df_test['text'].values, maxlen=MAX_TWEET_LENGTH, padding='post', truncating='post')

In [8]:
X_train_text.shape

(14483, 30)

In [8]:
text_input = tf.keras.layers.Input(shape=(MAX_TWEET_LENGTH,), name='article_body_input')
text_embed = tf.keras.layers.Embedding(biggest_token + 1, 50, input_length=MAX_TWEET_LENGTH, name='article_body_embedding')(text_input)
text_conv = tf.keras.layers.Conv1D(256, 10, name='article_body_conv')(text_embed)
text_pool = tf.keras.layers.GlobalMaxPool1D(name='article_body_pooling')(text_conv)
#concat = tf.keras.layers.concatenate([text_pool])
dense_100 = tf.keras.layers.Dense(100, activation='relu')(text_pool)
dense_50 = tf.keras.layers.Dense(50, activation='relu')(dense_100)
out_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense_50)
model = tf.keras.models.Model(inputs=[text_input], outputs=out_layer)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
article_body_input (InputLay [(None, 30)]              0         
_________________________________________________________________
article_body_embedding (Embe (None, 30, 50)            1420650   
_________________________________________________________________
article_body_conv (Conv1D)   (None, 21, 256)           128256    
_________________________________________________________________
article_body_pooling (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               25700     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
__

In [9]:
y_train = df_train['label'].values
y_test = df_test['label'].values
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, mode='max', restore_best_weights=True)

history = model.fit(X_train_text, y_train, epochs=100, batch_size=128, validation_data=(X_test_text, y_test), callbacks=[callback])

Train on 14483 samples, validate on 3781 samples
Epoch 1/100
14464/14483 [============================>.] - ETA: 0s - loss: 0.5082 - acc: 0.7436WARNING:tensorflow:From C:\Users\borin\anaconda3\envs\coursework\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
14483/14483 [==============================] - 5s 341us/sample - loss: 0.5078 - acc: 0.7438 - val_loss: 0.6439 - val_acc: 0.7249
Epoch 2/100
14483/14483 [==============================] - 5s 343us/sample - loss: 0.1652 - acc: 0.9327 - val_loss: 0.8939 - val_acc: 0.6414
Epoch 3/100
14483/14483 [==============================] - 5s 344us/sample - loss: 0.0493 - acc: 0.9838 - val_loss: 1.6173 - val_acc: 0.6133
Epoch 4/100
14483/14483 [==============================] - 5s 346us/sam

In [10]:
preds = model.predict(X_test_text)
model.evaluate(X_test_text, y_test)

[0.643860210008137, 0.7249405]

In [12]:
y_pred = []
for i in range(len(preds)):
    if(preds[i]<0.5):
        y_pred.append(0)
    else:
        y_pred.append(1)
    
y_pred = np.array(y_pred)

In [13]:
import sklearn.metrics as skl_m

f1 = skl_m.f1_score(y_test, y_pred)
f1

0.6927639383155397